In [4]:
# Generator

In [5]:
from math import sqrt, cbrt
# cbrt is only available in Python 3.11 and higher
# used Walrus Operator :=
lst = [
    (i, int(sq), int(cb))
    for i in range(2, 100)
    if (sq := sqrt(i)) == int(sq) and (cb := cbrt(i)) == int(cb)
]


In [7]:
lst

[(64, 8, 4)]

In [8]:
#Now, I want the first 5 numbers, but I have no idea what range I should use, so I'm going to try some large number:

In [9]:
lst = [
    (i, int(sq), int(cb))
    for i in range(2, 5_000_000)
    if (sq := sqrt(i)) == int(sq) and (cb := cbrt(i)) == int(cb)
]
lst

[(64, 8, 4),
 (729, 27, 9),
 (4096, 64, 16),
 (15625, 125, 25),
 (46656, 216, 36),
 (117649, 343, 49),
 (262144, 512, 64),
 (531441, 729, 81),
 (1000000, 1000, 100),
 (1771561, 1331, 121),
 (2985984, 1728, 144),
 (4826809, 2197, 169)]

In [10]:
# using generator comprehension

In [11]:
results = (
    (i, int(sq), int(cb))
    for i in range(2, 50_000_000)
    if (sq := sqrt(i)) == int(sq) and (cb := cbrt(i)) == int(cb)
)

In [12]:
extract = [next(results) for _ in range(10)]
extract


[(64, 8, 4),
 (729, 27, 9),
 (4096, 64, 16),
 (15625, 125, 25),
 (46656, 216, 36),
 (117649, 343, 49),
 (262144, 512, 64),
 (531441, 729, 81),
 (1000000, 1000, 100),
 (1771561, 1331, 121)]

In [13]:
from itertools import islice

In [14]:
results = (
    (i, int(sq), int(cb))
    for i in range(2, 50_000_000)
    if (sq := sqrt(i)) == int(sq) and (cb := cbrt(i)) == int(cb)
)
list(islice(results, 10))

[(64, 8, 4),
 (729, 27, 9),
 (4096, 64, 16),
 (15625, 125, 25),
 (46656, 216, 36),
 (117649, 343, 49),
 (262144, 512, 64),
 (531441, 729, 81),
 (1000000, 1000, 100),
 (1771561, 1331, 121)]

In [16]:
# We still have the issue of specifying that huge range - it works, but maybe I need to get the first 50 numbers - in which case that range may not be large enough - so we do not have general solution.
# So, in this case, a generator expression gives us a partially better solution than a list comprehension, but an even better solution can be achieved using a generator function.


In [17]:
def numbers():
    number = 2
    while True:
        sq = sqrt(number)
        cb = cbrt(number)
        if sq == int(sq) and cb == int(cb):
            yield number, int(sq), int(cb)
        number += 1

In [18]:
list(islice(numbers(), 10))

[(64, 8, 4),
 (729, 27, 9),
 (4096, 64, 16),
 (15625, 125, 25),
 (46656, 216, 36),
 (117649, 343, 49),
 (262144, 512, 64),
 (531441, 729, 81),
 (1000000, 1000, 100),
 (1771561, 1331, 121)]

In [19]:
# example using contexmanager 

In [20]:
from contextlib import contextmanager

In [21]:
@contextmanager
def echo():
    try:
        print("Entering context manager")
        yield lambda x: f"echo says: {x}"
    except Exception as ex:
        print(f"An exception occurred - we may want to handle it, or not: {ex}")
    finally:
        print("Exiting context manager - runs whether an exception occurred or not")

In [22]:
with echo() as func:
    print(func("hello"))
    print(func("bye"))

Entering context manager
echo says: hello
echo says: bye
Exiting context manager - runs whether an exception occurred or not


In [23]:
with echo() as func:
    print(func("hello"))
    raise ValueError("Wrong value")

Entering context manager
echo says: hello
An exception occurred - we may want to handle it, or not: Wrong value
Exiting context manager - runs whether an exception occurred or not


In [24]:
@contextmanager
def square():
    try:
        print("Entering context manager")
        yield lambda x: x * x
    except Exception as ex:
        print(f"An exception occurred - we may want to handle it, or not: {ex}")
    finally:
        print("Exiting context manager - runs whether an exception occurred or not")

with square() as func:
    print(func(10))



Entering context manager
100
Exiting context manager - runs whether an exception occurred or not


In [25]:
with square() as func:
    print(func('a'))

Entering context manager
An exception occurred - we may want to handle it, or not: can't multiply sequence by non-int of type 'str'
Exiting context manager - runs whether an exception occurred or not


In [26]:
# example using csv

In [27]:
import csv

headers = ["first_name", "last_name"]
data = [
    ("Isaac", "Newton"),
    ("Gottfried", "Leibniz"),
    ("Joseph", "Fourier"),
    ("John", "von Neumann"),
]

with open("test.csv", "w") as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(headers)
    for row in data:
        csv_writer.writerow(row)


In [28]:
#We can check the contens of that file:

In [29]:
with open("test.csv") as f:
    print(f.readlines())

['first_name,last_name\n', '\n', 'Isaac,Newton\n', '\n', 'Gottfried,Leibniz\n', '\n', 'Joseph,Fourier\n', '\n', 'John,von Neumann\n', '\n']


In [32]:
with open("test.csv") as f:
    csv_reader = csv.reader(f)
    for row in csv_reader:
        print(row)

['first_name', 'last_name']
[]
['Isaac', 'Newton']
[]
['Gottfried', 'Leibniz']
[]
['Joseph', 'Fourier']
[]
['John', 'von Neumann']
[]


In [30]:
#custom function

In [33]:
@contextmanager
def csv_reader(file_path):
    with open(file_path) as f:
        yield csv.reader(f)    

In [34]:
with csv_reader("test.csv") as reader:
    for row in reader:
        print(row)

['first_name', 'last_name']
[]
['Isaac', 'Newton']
[]
['Gottfried', 'Leibniz']
[]
['Joseph', 'Fourier']
[]
['John', 'von Neumann']
[]


In [35]:
# best solution for both read and write

In [36]:
@contextmanager
def csv_file(file_path, *, mode="r"):
    if mode == "r":
        with open(file_path) as f:
            yield csv.reader(f)
    elif mode in {'w', 'a'}:
        with open(file_path, mode) as f:
            yield csv.writer(f)
    else:
        raise ValueError("Unsuported mode - must be one of 'r', 'w', 'a'")

with csv_file("test3.csv", mode="w") as writer:
    writer.writerow(list('abc'))
    writer.writerow(list('def'))

with csv_file("test3.csv") as reader:
    for row in reader:
        print(row)



['a', 'b', 'c']
[]
['d', 'e', 'f']
[]
